# Performance Profiling Demo

## Setup
The execution time profiler is packaged with Python, and does not need to be installed separately.  However, the memory profiler is a python package that can be installed with a package installer like `pip`.

### Install memory profiler
The following command will load memory_profiler into your environment.

`pip install memory_profiler`

### Register memory profiler magic commands in the notebook
The following cell will load the memory profiler notebook extension to enable profiling magics.

In [ ]:
%load_ext memory_profiler

### Set up function inputs
The cell below contains demo-specific setup, including parameter variables that will be passed to scripts below.

In [ ]:
from phdi.azure import AzureFhirServerCredentialManager
from pathlib import Path
schema_path = Path("example_schema.yaml")  # Path to a schema config file.
output_path = Path("output")                    # Path to directory where tables will be written
output_format = "parquet"            # File format of tables
fhir_url = "https://pitest-fhir.azurehealthcareapis.com"           # The URL for a FHIR server
cred_manager = AzureFhirServerCredentialManager(fhir_url)

### Import library code to profile
For this demo, we'll load a few different versions of schemas.py, and load them to different module variables so we can reference them separately throughout the demo. This is only needed because we have different versions of the same code that we will be profiling.  Normally this would not be needed.

In [ ]:
import importlib
import sys
spec = importlib.util.spec_from_file_location("schemas-current", "schemas-0-current.py")
schemas_current = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_current)

spec = importlib.util.spec_from_file_location("schemas-evaluatefhirpath", "schemas-1-evaluatefhirpath.py")
schemas_evaluatefhirpath = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_evaluatefhirpath)

spec = importlib.util.spec_from_file_location("schemas-uncachedpathcompile", "schemas-2-uncachedpathcompile.py")
schemas_uncachedpathcompile = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_uncachedpathcompile)

spec = importlib.util.spec_from_file_location("schemas-fulltableextract", "schemas-3-fulltableextract.py")
schemas_fulltableextract = importlib.util.module_from_spec(spec)
spec.loader.exec_module(schemas_fulltableextract)


## Execution Speed Profiling
There are a few code profile utilities available - we'll use [cProfile](https://docs.python.org/3/library/profile.html).

### Current code execution

In [ ]:
import cProfile
import pstats

cProfile.run('schemas_current.make_schema_tables(schema_path, output_path / "current", output_format, fhir_url, cred_manager)',"output/current/cProfile-output.txt")


In [ ]:
pstats.Stats("output/current/cProfile-output.txt").sort_stats(pstats.SortKey.CUMULATIVE).print_stats()

### Evaluate FHIRPath execution

In [ ]:
import cProfile
import pstats

cProfile.run('schemas_evaluatefhirpath.make_schema_tables(schema_path, output_path / "evaluatefhirpath", output_format, fhir_url, cred_manager)',"output/evaluatefhirpath/cProfile-output.txt")


In [ ]:
pstats.Stats("output/evaluatefhirpath/cProfile-output.txt").sort_stats(pstats.SortKey.CUMULATIVE).print_stats()

### Separate FHIRPath parse + apply

In [ ]:
import cProfile
import pstats

cProfile.run('schemas_uncachedpathcompile.make_schema_tables(schema_path, output_path / "uncachedpathcompile", output_format, fhir_url, cred_manager)',"output/uncachedpathcompile/cProfile-output.txt")


In [ ]:
pstats.Stats("output/uncachedpathcompile/cProfile-output.txt").sort_stats(pstats.SortKey.CUMULATIVE).print_stats()

## Memory Profiling
There are a few options for memory profiling in python.  The demo below uses the [memory_profiler](https://pypi.org/project/memory-profiler/)
The memory_profiler python package can be run in a few ways:
* You can run the following from the command line: 
  
  `mprof run --python python my-python-script.py`
  
  This will collect function-specific data, as well as create a file with data that can be displayed as a graph using `mprof plot`

* You can see line-by-line at the command line.  First, add the `@profile` decorator to any functions you would like to analyze, and then run
  
  `python -m memory_profiler my-python-script.py`

  This will print a line-by-line analysis of memory consumption for the targets of the `@profile` decorator.

* Alternatively, you can use the notebook extention, with magic:

  `%mprun -f target_function my-python-script.py`.  
  
  The -f parameter may be used to specify the function you'd like to profile without adding the `@profile` decorator.

We will take a look at profiles for 2 different architectural approaches.  One which incrementally requests and writes to a file, and another that collects data from all requests, and writes the entire subset to a file.  We'll look at the out put both line-by-line and over the execution of the program.

### Line-by-line memory profiling
#### Full table extract

In [ ]:
!python -m memory_profiler schemas-3-fulltableextract.py example_schema.yaml output/fullextract parquet https://pitest-fhir.azurehealthcareapis.com

#### Incremental extract

In [ ]:
!python -m memory_profiler schemas-0-current.py example_schema.yaml output/current parquet https://pitest-fhir.azurehealthcareapis.com

### Timeline memory profiling
#### Full table extract

In [ ]:
!mprof run --output fulltableextract_mprofile.dat --python python schemas-3-fulltableextract.py example_schema.yaml output/fullextract parquet https://pitest-fhir.azurehealthcareapis.com

#### Incremental extract

In [ ]:
!mprof run --output current_mprofile.dat --python python schemas-3-current.py example_schema.yaml output/fullextract parquet https://pitest-fhir.azurehealthcareapis.com

### Notebook memory profiling

In [ ]:
%mprun -f schemas_fulltableextract.demo_run schemas_fulltableextract.demo_run(schema_path, output_path / "fulltableextract", output_format, fhir_url)